In [3]:
import os, glob
import numpy as np
from skimage import io, exposure, img_as_ubyte, segmentation
import xtiff

def merge_masks_phase(data_path, output_path, pipeline_name, plateID, well, field_num):
    p_src = os.path.join(data_path, plateID,"Analysis/registered_stacks/",plateID+"_P_"+well+"_"+str(field_num)+"_reg_stack.tif")
    sys.stdout.write("Processing 2 channel video for "+p_src+"\n")
    p_raw = io.imread(p_src)
    p_stack = img_as_ubyte(exposure.rescale_intensity(p_raw))

    cyto_mask_src = os.path.join(output_path, plateID,"Analysis",pipeline_name,"intermediate_files/tracking",well,"field_"+str(field_num),"cyto_filtered_masks")
    cyto_mask_filenames = sorted(glob.glob(cyto_mask_src+"/mask*"))

    cyto_mask_list = []
    for fn in cyto_mask_filenames:
        #read in the mask image
        masks = io.imread(fn)%256 #relabel to fit in 8 bits
        cyto_mask_list.append(masks)
    cyto_mask_stack  = np.stack(cyto_mask_list).astype("uint8")

    pm_stack = np.stack((cyto_mask_stack, p_stack), axis = 1) #TCCYX

    output_subdir = os.path.join(output_path, plateID, "Analysis", pipeline_name, "QA")
    os.makedirs(output_subdir, exist_ok=True)
    filename = os.path.join(output_subdir,plateID+"_2CH_"+well+"_"+str(field_num)+"_reg_stack.ome.tiff")
    #save a copy of the phase stack to RDS as a debug tool
    res = xtiff.to_tiff(pm_stack, filename)
    sys.stdout.write("Saved 2 channel video at "+filename+"\n")

    
data_path = "/home/exacloud/gscratch/HeiserLab/images/"
output_path = "/home/groups/heiserlab_genomics/home/dane/CellTracking/images/"
#plateID = "HC01601"
pipeline_name = "CcntB"
well = "B2"
field_num = 1

plateIDs = ["HC01701", "HC01801", "HC01901", "HC02001"]
for plateID in plateIDs:
    res = merge_masks_phase(data_path, output_path, pipeline_name, plateID, well, field_num)
    